In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re
import time
from urllib.request import Request, urlopen
from urllib.parse import urljoin
from urllib.request import urlopen

In [6]:
def firefox_request_soup(link):
    site= link
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    req = Request(site,headers=hdr)
    try:
        resp = urlopen(req)
    except Exception as e:
        print("returned exception", e, "on link", link)
        return None
    soup = BeautifulSoup(resp, features='lxml')
    return(soup)

def get_links(input_page, text):
    # gets all links on the page that contain the given text in the URL
    input_soup = firefox_request_soup(input_page)

    if input_soup == None:
        return []

    links = []

    for i in input_soup.find_all("a", href=True):
        if "regex=" in text:
            regex = re.compile(text.lstrip("regex="))
            if re.match(regex, i['href']):
                links.append(i['href'])
        elif text in i['href']:
            links.append(i['href'])

    links = list(set(links))
    full_links = [urljoin(input_page, link) for link in links]
    return full_links

In [36]:
links

['https://www.federalreserve.gov/releases/lbr/20110630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20060331/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20181231/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20201231/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20080630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20121231/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20050630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20150630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20140630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20151231/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20180331/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20050331/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20040630/default.htm',
 'https://www.federalreserve.gov/releases/lbr/20081231/default.htm',
 'https://www.federalreserve.gov/r

In [52]:
def get_df(input_page):
    # get table
    tables = pd.read_html(input_page)
    lengths = [len(table) for table in tables]
    index = lengths.index(max(lengths))
    df = tables[index]
    
    # get date
    if "current" in input_page:
        date = '20210630'
    else:
        date = re.search(r'\d{8}', input_page).group(0)
    df["date"] = date
    
    return(df)


In [53]:
links = get_links("https://www.federalreserve.gov/releases/lbr/",
         "default.htm")
links = [i for i in links if 'lbr' in i]

In [56]:
dfs = list()
for link in links:
    result = get_df(link)
    dfs.append(result)

In [57]:
df_master = pd.concat(dfs)
df_master

,Bank Name / Holding Co Name,Nat'l Rank,Bank ID,Bank Location,Charter,Consol Assets (Mil $),Domestic Assets (Mil $),Pct Domestic Assets,Pct Cumulative Assets,Domestic Branches,...,date,Natl Rank,Consol Assets,Domestic Assets,Pct DomesticAssets,Pct CumulativeAssets,Pct ForeignOwned,Pct Domestic Assets,Pct Cumulative Assets,Pct Foreign Owned
0,JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO,1.0,852218.0,"COLUMBUS, OH",NAT,1791060,1117246,62,15,5320,...,20110630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BANK OF AMER NA/BANK OF AMER CORP,2.0,480228.0,"CHARLOTTE, NC",NAT,1454051,1341798,92,28,5795,...,20110630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CITIBANK NA/CITIGROUP,3.0,476810.0,"LAS VEGAS, NV",NAT,1216291,600156,49,38,1024,...,20110630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WELLS FARGO BK NA/WELLS FARGO & CO,4.0,451965.0,"SIOUX FALLS, SD",NAT,1104833,1056663,96,48,6379,...,20110630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U S BK NA/U S BC,5.0,504713.0,"CINCINNATI, OH",NAT,310100,308746,100,51,3135,...,20110630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,NEWDOMINION BK/,1779.0,3299814.0,"CHARLOTTE, NC",SNM,301,301,100,100,1,...,20150930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1788,JONAH BK/JONAH BSHRS,1780.0,3412417.0,"CASPER, WY",SMB,301,301,100,100,3,...,20150930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1789,UNITED BK OF UNION/CARDINAL BC II,1781.0,562058.0,"UNION, MO",SNM,300,300,100,100,2,...,20150930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1790,ALVA ST B&TC/GRACE INV CO ESOP,1782.0,246956.0,"ALVA, OK",SNM,300,300,100,100,3,...,20150930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df_master.to_csv('../data/bank_assets.csv')